## Experiment testing v2.0 with live streaming data and realtime processing, prediction and make change to dot

In [1]:
# code for storing subject and trial info
from psychopy import gui, visual, core, data, event, logging, clock, colors, layout
# GUI for saving data # Store info about the experiment session
expName = 'closedloop'
exType = 'wet'
expInfo = {'participant': 'X00','type': exType}
dlg = gui.DlgFromDict(dictionary=expInfo, sortKeys=False, title=expName)
if dlg.OK == False:
    core.quit()  # user pressed cancel

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\shaya\anaconda3\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [2]:
import numpy as np
import pandas as pd
import realtime_utilsH as utils
import torch

#INIT
filt_ord = 2
freq_limits = np.asarray([[1,100]]) 
freq_limits_names = ['1_100Hz']
sample_duration = 125
sampling_frequency = 250
electrode_names =  ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
filters = utils.init_filters(freq_limits, sampling_frequency, filt_type = 'bandpass', order=filt_ord)
segments, labels, predictions = [], [], []
total_outlier = 0

In [3]:
# init DL model
#NOTE: Update path for model later!!
subject = expInfo['participant']
net = utils.EEGNET()
path = r'F:\Academics\M2 CNN\Semester 2\BMI Lab Elche\Project\Final trials\old files\EEGNET_Finetuned_X02' # update path later
net.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
net = net.float()
net.eval()

EEGNET(
  (temporal): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=same, bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (spatial): Sequential(
    (0): Conv2d(8, 16, kernel_size=(8, 1), stride=(1, 1), groups=8, bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=True)
  )
  (seperable): Sequential(
    (0): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=same, groups=16, bias=False)
    (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), padding=same, bias=False)
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=True)
  )
  (avgpool1): AvgPool2d(kernel_size=[1, 5], stride=[1, 5], padding=0)
  (avgpool2): AvgPool2d(kernel_size=[1, 5], stride=[1, 5], padding=0)
  (dropout): Dropout(p=0.25, inplace=False)
  (view): Sequential(
    (0): Flatten()
  )
  (fc2): Li

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#psychopy libraries for running the visual cues
from psychopy import gui, visual, core, data, event, logging, clock, colors, layout
import psychopy.iohub as io
from psychopy.hardware import keyboard
#numpy and pd for data storing and manipulation
import numpy as np
import pandas as pd
from numpy.random import  shuffle
# misc libraries to structure the cues properly and save it with date time and stuff
import time
from datetime import date
from pathlib import Path
import random
import os
import time
import msvcrt
import datetime
# lab streaming layer library to capture the data sent by unicorn EEG headset
from pylsl import StreamInlet, resolve_stream

# change path of folders according to your needs
# Data file name stem = absolute path + name; later add .psyexp, .csv, .log, etc
result_path = Path(f'./Expdata/Subjects/'+exType+'/'+expInfo['participant']+'/'+expName+'/')
result_path.mkdir(exist_ok=True, parents=True)

columns=['Time','FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8','AccX','AccY','AccZ','Gyro1','Gyro2','Gyro3',
                                  'Battery','Counter','Validation']

data_dict = dict((k, []) for k in columns)
current_seg = pd.DataFrame()
total_MI_outliers = 0
all_MI_segments, all_MI_labels, predictions = [], [], []

# time for the trial
calTime = 10.0
restTime = 3.0
cueTime = 2.0
focusTime = 10.0
blkTime = 5.0

# --------- Preparing Ready Window --------
win = visual.Window(
    size=(1440, 900), fullscr=True, screen=1, 
    winType='pyglet', allowGUI=False, allowStencil=False,
    monitor='testMonitor', color='black', colorSpace='rgb',
    blendMode='avg', useFBO=True, 
    units='height')

# -----------Initializing stimuli to be shown -------
# Initialize components for Routine "10 sec calibration"
ten_sec = ten_sec = visual.ShapeStim(
    win=win, name='ten_sec',color = 'black',
    size=(0.044, 0.044), vertices='circle', # change size to 0.045,0.045 if zoomed in
    ori=0.0, pos=(0, 0), anchor='center',
    lineWidth=1.0,     colorSpace='rgb',  lineColor='darkgrey', fillColor=(0.3255,0.3255,0.3255),
    opacity=None, depth=-2.0, interpolate=True)

# Initialize components for Routine "trial"
# image of cross being showed
restCross = visual.ImageStim(
    win=win, name='RestCross',
    image='VC_Cross.jpg', mask=None, anchor='center',
    ori=0.0, pos=(0, 0), size=None,
    color=[1,1,1], colorSpace='rgb', opacity=None,
    flipHoriz=False, flipVert=False,
    texRes=128.0, interpolate=True, depth=-1.0)
# this has been been manipulated to show random cues for the subject throughout the trial
Cue = visual.ImageStim(
    win=win, name='Cue',
    image='VC_Right.jpg', mask=None, anchor='center',
    ori=0.0, pos=(0, 0), size=None,
    color=[1,1,1], colorSpace='rgb', opacity=None,
    flipHoriz=False, flipVert=False,
    texRes=128.0, interpolate=True, depth=-1.0)
# the small dot on the screen where the subject has to focus for our trial, later to be move during closed loop trials
focus = visual.ShapeStim(
    win=win, name='focus',color = 'black',
    size=(0.044, 0.044), vertices='circle', # change size to 0.045,0.045 if zoomed in
    ori=0.0, pos=(0, 0), anchor='center',
    lineWidth=1.0,     colorSpace='rgb',  lineColor='darkgrey', fillColor=(0.3255,0.3255,0.3255),
    opacity=None, depth=-2.0, interpolate=True)
# blank screen for rest between cues for blinking, swallowing and other stuff
Blank = visual.ImageStim(
    win=win, name='BlankScreen',
    image='VC_Blank.jpg', mask=None, anchor='center',
    ori=0.0, pos=(0, 0), size=None,
    color=[1,1,1], colorSpace='rgb', opacity=None,
    flipHoriz=False, flipVert=False,
    texRes=128.0, interpolate=True, depth=-1.0)

# below code is for initializing the streaming layer which will help us capture data later
finished = False
streams = resolve_stream()
inlet = StreamInlet(streams[0])

# Auto updating trial numbers
trial_list = []
for instance in os.scandir(result_path):
        if instance.path.endswith('.csv'):
            length = len(instance.path)
            trial_list.append(int(instance.path[length-5]))

if len(trial_list) == 0:
    session = '01'
elif len(trial_list) < 9 :
    session = len(trial_list) + 1
    session = '0' + str(session)
else :
    session = str(len(trial_list) + 1)
    
    
print(f"Conducting {expName} experiment for subject :", expInfo['participant'])
print('No. of Practice Trials before :', 2)
print("Trial Number :", session)

print('Actual Trial')
print('Total number of trials as of now :', int(session) + 2)
results_fname = expInfo['participant']+'_'+str(date.today())+'_'+expName+'_'+ expInfo['type']+'_'+session+'.csv'
print("Saving file as .. ", results_fname)

# -------- Beginning of trial ----------
# Create a stimulus for a certain window
readyText = visual.TextStim(win, "Ready?", color=(1,1,1))
readyText.draw()
#present ready text on the screen 
win.flip()
#wait for user to press return key 
event.waitKeys(keyList=['return'])

# image list with labels for showing randomly and storing in the database
# creating cue list
img_list = [('VC_Relax.jpg',0),('VC_Right.jpg',1),('VC_Left.jpg',2)]*2
trials = len(img_list)
np.random.shuffle(img_list)
# calculating run time to shut off data capturing
runtime = calTime +trials*(restTime + cueTime + focusTime + blkTime)

classes = [] # to store the showed classes in a list later to be added to database .csv file
Fs = 250 # sampling frequency of Unicorn EEG cap
temp = []
times = []
#start = time.time()
flush = [17*Fs, 37*Fs, 57*Fs, 77*Fs, 97*Fs, 117*Fs]

initial = 0
final = 125
prediction = -1

while not finished:
    
    sample, timestamp = inlet.pull_sample()
    
    if len(times) ==0 :
        start_count = sample[15]
        ten_sec.draw()
        win.flip()
        core.wait(calTime)    
        classes = classes + 10*250*['Y']
        
    elif len(times) == Fs*10:
    
        for cue in img_list:
            
            pos = (0,0)
            size = (0.045, 0.045)
            focus.setPos(pos)
            focus.setSize(size)  
            
            Cue.image = cue[0] 
            cue_cls = cue[1] 
            
            restCross.draw()
            win.flip()
            core.wait(restTime)
            
            Cue.draw()
            win.flip()
            core.wait(cueTime)
            
            focus.draw()
            win.flip()
            core.wait(2)
            
            #flushing data to do real time stuff
            while sample[15] not in [a + start_count for a in flush]:
                sample, timestamp = inlet.pull_sample()
                res = [timestamp] + sample
                data_dict = utils.update_data(data_dict,res)
                times.append(timestamp)
                
                if len(data_dict['FZ']) % 125 == 0:
                    df, initial, final = utils.segment_dict(initial, final, sample_duration, data_dict)
                    segment_filt, outlier, filters = utils.pre_processing(df, electrode_names, filters, 
                                    sample_duration, freq_limits_names, sampling_frequency)
                    current_seg = utils.concatdata(current_seg,segment_filt)
            # all columns are same now
            labels = []
            #for num in utils.Genrandom(int((focusTime-2)//0.5)):
            for num in range(0,int((focusTime-2)//0.5)):
                # inlet stuff, preprocess and predict
                buffer = {'time':[],'sample':[]}
                while len(buffer['sample'])<125:
                    sample, timestamp = inlet.pull_sample()
                    labels.append(cue_cls)
                    buffer['time'].append(timestamp)
                    buffer['sample'].append(sample)
                    
                    res = [timestamp] + sample 
                    data_dict = utils.update_data(data_dict,res)
                    times.append(timestamp)
                    
                    if len(data_dict['FZ']) % 125 == 0:
                        df, initial, final = utils.segment_dict(initial, final, sample_duration, data_dict)
                        segment_filt, outlier, filters = utils.pre_processing(df, electrode_names, filters, 
                                        sample_duration, freq_limits_names, sampling_frequency)
                        current_seg = utils.concatdata(current_seg,segment_filt)      
                # do prediction with current segment and update number
                if len(labels) >= 500:
                    labels_df = pd.DataFrame(labels, columns = ['label'])
                    MI_state, current_label = utils.is_MI_segment(labels_df)
                    if MI_state:
                        if outlier > 0:
                            total_MI_outliers += 1
                            print('OUTLIER')
                        else:
                            all_MI_segments.append(current_seg)
                            all_MI_labels.append(int(current_label)) 
                            prediction = utils.do_prediction(current_seg, net)
                            predictions.append(int(prediction[0]))
                            print(f"prediction: {prediction}, true label: {current_label}")
                    else:
                        print(current_label)

                #TODO here call movedotwhen instead of movedot            
                focus, pos, size = utils.movedot(prediction,focus,pos,size)  
                #focus, pos, size = utils.movedot(num,focus,pos,size) 
                focus.draw()
                win.flip()
                core.wait(0.45)
            
            Blank.draw()
            win.flip()
            core.wait(blkTime)
            
            # updating class list based on the cues shown
            if cue_cls == 0:
                temp = 3*Fs*['Z']+4*Fs*['relax']+8*Fs*[0]+5*Fs*['rest']
                classes = classes + temp
            elif cue_cls == 1:
                temp = 3*Fs*['Z']+4*Fs*['right']+8*Fs*[1]+5*Fs*['rest']
                classes = classes + temp
            elif cue_cls == 2:
                temp = 3*Fs*['Z']+4*Fs*['left']+8*Fs*[2]+5*Fs*['rest']
                classes = classes + temp

        message = visual.TextStim(win, text="Trial Done")
        message.draw()
        win.flip()
        core.wait(5.0)
        win.close() 
        
   # ending trial after runtime gets over (calculated beforehand)
    if len(times) > runtime*Fs or len(times) == runtime*Fs :
        finished = True
        break
        
    # updating data dictionary with newly transmitted samples 
    res = [timestamp] + sample
    data_dict = utils.update_data(data_dict,res)
    times.append(timestamp)
    
    
    if len(data_dict['FZ']) % 125 == 0:
        df, initial, final = utils.segment_dict(initial, final, sample_duration, data_dict)
        segment_filt, outlier, filters = utils.pre_processing(df, electrode_names, filters, 
                        sample_duration, freq_limits_names, sampling_frequency)
        current_seg = utils.concatdata(current_seg,segment_filt)
    
data_dict['Class'] = classes
# making dictionary into a dataframe for saving it as csv
record_data = pd.DataFrame.from_dict(data_dict)

#fname = Path('./Expdata/Subjects/'+expInfo['participant']+'/'+ expName + '/'+results_fname)
fname = Path('./Expdata/Subjects/'+exType+'/'+expInfo['participant']+'/'+expName+'/'+results_fname)
record_data.to_csv(fname, index = False)
print('Trial Ended')

#TODO save all_MI_segments, all_MI_labels, predictions

C:\Users\shaya\anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Conducting closedloop experiment for subject : X00
No. of Practice Trials before : 2
Trial Number : 04
Actual Trial
Total number of trials as of now : 6
Saving file as ..  X00_2022-05-27_closedloop_wet_04.csv
125
125
125
250
125
375
125
500
OUTLIER
125
625
OUTLIER
125
750
OUTLIER
125
875
OUTLIER
125
1000
OUTLIER
125
1125
OUTLIER
125
1250
OUTLIER
125
1375
OUTLIER
125
1500
OUTLIER
125
1625
OUTLIER
125
1750
OUTLIER
125
1875
OUTLIER
125
2000
OUTLIER
125
125
125
250
125
375
125
500
OUTLIER
125
625
OUTLIER
125
750
OUTLIER
125
875
OUTLIER
125
1000
OUTLIER
125
1125
OUTLIER
125
1250
OUTLIER
125
1375
OUTLIER
125
1500
OUTLIER
125
1625
OUTLIER
125
1750
OUTLIER
125
1875
OUTLIER
125
2000
OUTLIER
125
125
125
250
125
375
125
500
OUTLIER
125
625
OUTLIER
125
750
OUTLIER
125
875
OUTLIER
125
1000
OUTLIER
125
1125
OUTLIER
125
1250
OUTLIER
125
1375
OUTLIER
125
1500
OUTLIER
125
1625
OUTLIER
125
1750
OUTLIER
125
1875
OUTLIER
125
2000
OUTLIER
125
125
125
250
125
375
125
500
OUTLIER
125
625
OUTLIER
125
750
OUTL

In [5]:
all_MI_segments

[]

In [6]:
all_MI_labels

[]

In [7]:
current_label

1